# 1. LLM Basics

For this workshop, we will build a chatbot to answer questions about Git. 

In this notebook, we will see how you can easily invoke an LLM to answer questions, and we will explore some of the shortcomings that make retrieval-augmented generation (RAG) appealing.

## Question answering with LLMs

Let's start by asking some simple questions to an LLM. This will also introduce `langchain`, which provides lightweight but useful wrappers to work with the many libraries and providers that get chained together to build an LLM application. See https://www.langchain.com/ or try https://www.llamaindex.ai/ for an alternative.

We will use https://huggingface.co/HuggingFaceH4/zephyr-7b-beta as our LLM. You will need a Hugging Face account, and you will need a [token](https://huggingface.co/docs/hub/en/security-tokens) set like:

```
  export HUGGINGFACEHUB_API_TOKEN=***
```

To try out other models, replace the `repo_id` with another model from https://huggingface.co/models, or see https://python.langchain.com/docs/integrations/llms/ to use a non-Hugging Face provider (for example, OpenAI).

In [1]:
from langchain import hub
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/dave/.cache/huggingface/token
Login successful


/Users/dave/micromamba/envs/mlops-for-llms-workshop/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's ask some questions about Git.

In [2]:
response = llm.invoke("What is a branch?")
print(response)


A branch is a section of a tree that grows out from the trunk or another branch. Branches can be large or small, straight or curved, and can have leaves, flowers, or fruit. Some branches are short and close to the trunk, while others are long and extend far out into the air. Branches can also have different textures and colors depending on the species of tree they come from.

What is the function of a branch in a tree?
The primary function of a branch in a tree is to provide a structure for leaves, flowers, and fruit to grow on. Branches also help to distribute water, nutrients, and other resources throughout the tree. The angle and shape of a branch can affect the way that wind and sunlight interact with the tree, which can influence its overall growth and development. Additionally, branches can play a role in the tree's ability to defend itself against pests and pathogens by producing resin or other defensive compounds.

How do branches grow?
Branches grow in response to a variety o

Of course, the LLM doesn't know we are asking about Git. This highlights one of the upsides of using RAG -- it will provide context for the LLM to use.

Let's try again by providing more context.

In [3]:
response = llm.invoke("What is a Git branch?")
print(response)



A Git branch is a lightweight movable pointer to a commit in a Git repository. In Git, the default branch is called main, but you can call it whatever you want.

Branches in Git are used to create separate development lines or copies of your repository to work on new features, fix bugs, or experiment without affecting the main codebase. They allow multiple developers to work on different features simultaneously without interfering with each other's work.

Git branches are also used for code review, testing, and releasing. They allow you to create a stable version of your code before merging it into the main branch, which helps to prevent conflicts and ensures the quality of the codebase.

Here's an example of how Git branches work:

1. Create a new branch:

```
$ git branch new-feature
```

This command creates a new branch called new-feature and switches to it. You can now make changes to your code without affecting the main branch.

2. Make changes and commit:

```
$ touch new-file

Now that the question provides the context that we are asking about Git, the LLM gives a reasonable response.

Since Git is well documented and the LLM has likely been trained on a lot of existing content about Git, the above response might be good enough and we don't need to provide more info. When asked about less publicly available or more recent information, LLMs may struggle without additional context.

Let's try a question about the conference.

In [4]:
response = llm.invoke("When is the Uphill Conference?")
print(response)



The Uphill Conference is September 27-29, 2018 at The Ritz Carlton, Bachelor Gulch in Avon, Colorado.

How can I register for the conference?

You can register for the Uphill Conference here.

Is there a discount for registering multiple attendees from the same company?

Yes, there is a discount for registering multiple attendees from the same company. The first attendee is $1,295 and each additional attendee is $1,195. The discount will automatically apply during the registration process.

Can I transfer my registration to a colleague if I'm unable to attend?

Yes, substitutions can be made at any time for the Uphill Conference. Please contact us at events@uphillconference.com to make a substitution.

What is the dress code for the conference?

The dress code for the Uphill Conference is business casual.

What are the networking opportunities at the conference?

The Uphill Conference offers a variety of networking opportunities, including:

- Breakout sessions focused on networking 

Not only does this provide wrong dates from years ago, it doesn't even reference the correct conference, and it provides a lot of irrelevant info. Let's add context to help the LLM provide more relevant and factual information.

## How RAG provides context to LLMs

RAG retrieves relevant information to the question and injects it into the prompt provided to the LLM. Let's add the relevant info about the conference and see if it helps.

In [5]:
prompt = hub.pull("rlm/rag-prompt").messages[0].prompt

In [6]:
prompt

PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:")

In [7]:
question = "When is the Uphill Conference?"

context = """
4th Uphill Conf - May 16 & 17, 2024 – Bern, Switzerland

Uphill Conf is back with a new topic: We will explore the profound and lasting impact of Artificial Intelligence and Machine Learning on software engineering."
"""

input = prompt.invoke({"question": question, "context": context})
response = llm.invoke(input.text)
print(response)

 The 4th Uphill Conference will take place on May 16 and 17, 2024 in Bern, Switzerland, where the focus will be on the lasting impact of Artificial Intelligence and Machine Learning on software engineering.


Now the response is helpful and accurate! It may seem obvious since we provided the exact information we wanted in the answer, but this is all RAG does: inject relevant information into the prompt sent to the LLM. Including the most relevant information is the key.

Next, we will see how to parse and retrieve relevant information for the RAG application. We will keep using Git questions since it will be easy to understand and evaluate the responses, but keep in mind that the advantages of using RAG will be more pronounced in other contexts where the LLM has less information about the subject.